 
# <span style="color:purple">GIS and Machine Learning for Object Detection in Satellite Imagery</span>

<img src="img/python.png"></img>

## <span style="color:blue">Step 1: Data Preparation</span>

#### Goal

The goal of this step is to create the data that we need to train a Convolutional Neural Network (CNN) to learn how to detect CAFO sites in NAIP satellite imagery. 

Our CNN will be trained using extracted PNG images of NAIP satellite imagery over known CAFO locations.

#### Description

Using an ArcGIS Pro project, we will leverage two data sources to build our model inputs: a NAIP imagery service and a feature class containing identified CAFO sites in Kentucky (gis_inputs/ky_afo_lonx_nonzero) used in a image classification workshop.

This process iterates on each record of the Kentucky feature class containing CAFO sites, loading the NAIP imagery at the location at three different specified scales: 1:1000; 1:2000; and 1:3000, and exporting each as a .JPEG image in a designated directory. A total of 250 locations are used, resulting in 750 input features (3 for each location) for training and testing.

#### Pseudocode

1. Use an ArcGIS Pro Project with a template that includes:
     - NAIP Imagery Layer
     - CAFO Sites Layer
     - Borderless layout from map
2. Set references to:
     - Project
     - map
     - cafos_lyr
     - layout
     - Extents needed (python list?)
     - Output images directory
3. Iterate on each cafos_lyr record. In the iteration, perform the following:
     - Change the extent of the layout to the current site.
     - For each scale needed:
         - Set the camera scale to the scale
         - Export to JPEG to the output images directory using formatting for naming

In [ ]:
# Import needed modules
import arcpy
import os

In [ ]:
# Set reference to an ArcGIS Pro Project
aprx = arcpy.mp.ArcGISProject(r"C:\Users\albe9057\Documents\ANieto_SolutionEngineering\Projects\EPA\ArcGISPythonAPI_for_EPA\Work\EPA_CAFO_Training\EPA_CAFO_Training.aprx")

In [ ]:
# Set reference to our map, cafos layer, a layout object, and an output directory
m = aprx.listMaps()[0]
cafos_lyr = m.listLayers("ky_afo_lonx_nonzero")[0]
cafos_lyr_id_field = "FID"
lyt = aprx.listLayouts()[0]
mf = lyt.listElements()[0]
output_dir = r"C:\Users\albe9057\Documents\ANieto_SolutionEngineering\Projects\EPA\ArcGISPythonAPI_for_EPA\Inputs\cafo_image_training"

In [ ]:
# Set reference to a list of scales that will be used for image extraction
cafos_scales_list = [1000, 2000, 3000]

In [ ]:
# Main iteration loop
def main(
    aprx_obj,
    map_obj,
    lyr_obj,
    lyr_id_field,
    layout_obj,
    mapFrame_obj,
    output_image_dir,
    scales_list=[1000],
    projection_wkid="3857"):

    feature_count = arcpy.GetCount_management(lyr_obj).getOutput(0)

    arcpy.AddMessage("Extracting {0} images at specified scales for object labeling...".format(feature_count))

    counter = 0

    with arcpy.da.SearchCursor(lyr_obj, ['SHAPE@', lyr_id_field]) as cursor:
        for row in cursor:
            counter+=1
            arcpy.AddMessage("Extracting image {0} of {1}...".format(counter, feature_count))

            extent = row[0].extent.projectAs(projection_wkid)

            mapFrame_obj.camera.setExtent(extent)

            for scale in scales_list:
                mapFrame_obj.camera.scale = scale

                image_name = "img_{0}_{1}".format(str(row[1]), str(scale))
                layout_obj.exportToJPEG(os.path.join(output_image_dir, image_name))

    arcpy.AddMessage("Extraction completed.")

In [ ]:
# Execution
main(aprx, 
     m, 
     cafos_lyr, 
     cafos_lyr_id_field, 
     lyt, 
     mf, 
     output_dir, 
     cafos_scales_list)